
# 1 - Imports and defining functions

In [1]:
import meshplot as mp
import numpy as np

from pyFM.mesh import TriMesh


def plot_mesh(myMesh, cmap=None):
    mp.plot(myMesh.vertlist, myMesh.facelist, c=cmap)


def double_plot(myMesh1, myMesh2, cmap1=None, cmap2=None):
    d = mp.subplot(myMesh1.vertlist, myMesh1.facelist, c=cmap1, s=[2, 2, 0])
    mp.subplot(myMesh2.vertlist, myMesh2.facelist, c=cmap2, s=[2, 2, 1], data=d)


def visu(vertices):
    min_coord, max_coord = np.min(vertices, axis=0, keepdims=True), np.max(vertices, axis=0, keepdims=True)
    cmap = (vertices - min_coord) / (max_coord - min_coord)
    return cmap


# 2- Loading and processing a mesh

### Basic Mesh methods

A TriMesh class can be created from a path (to a .off or a .obj file) or simply an array of vertices and an optional array of faces.

The mesh can be centered, area-normalized, rotated or translated when loading.


Vertices and faces are stored in the 'vertlist' and 'facelist' attributes. One can also use 'mesh.vertices' and 'mesh.faces' to access them. While these notations can feel non-intuitive they result in clearer functions as it avoids expressions of the form ```mesh.vertices - vertices```.

A TriMesh class possess multiple attributes like edges, per-face area, per-vertex area, per-face normals, per-vertex normals, ...

In [2]:
mesh1 = TriMesh('data/lion-00.off', area_normalize=True, center=False)
mesh2 = TriMesh('data/cat-00.off', area_normalize=True, center=False)
# pcd2 = TriMesh(np.load('data/cat-00.npy'))


In [ ]:
mesh1.vertlist.shape

(5000, 3)

In [ ]:
plot_mesh(mesh1)
plot_mesh(mesh2)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-1.359730…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.001617…

In [ ]:
# # Attributes are computed on the fly and cached
# edges = mesh1.edges
#
# area = mesh1.area
#
# face_areas = mesh1.face_areas
# vertex_areas = mesh1.vertex_areas
# face_normals = mesh1.normals
#
# # AREA WEIGHTED VERTEX NORMALS
# vertex_normals_a = mesh1.vertex_normals
#
# # UNIFORM WEIGHTED VERTEX NORMALS
# mesh1.set_vertex_normal_weighting('uniform')
# vertex_normals_u = mesh1.vertex_normals

### Geodesics

We propose three versions to compute geodesics :
- Heat method - based on [potpourri3d](https://github.com/nmwsharp/potpourri3d) using robust laplacian (recommended)
- Heat method - pure python implementation from pyFM (not robust but control on the whole code)
- Dijkstra

In [ ]:
# Geodesic distance from a given index
# Set robust to False to obtain result from the Python implementation
dists = mesh1.geod_from(1000, robust=True)

In [ ]:
S1_geod = mesh1.get_geodesic(verbose=True)

  0%|          | 0/5000 [00:00<?, ?it/s]

In [ ]:
S1_geod.shape

(5000, 5000)

### Laplacian and functions

The spectrum of the LBO can be computed easily.

Eigenvalues and eigenvectors are stored in the ```mesh.eigenvalues``` and ```mesh.eigenvectors``` attributes.

Gradient and divergence can be computed using the associated methods. Using the ```mesh.project``` and ```mesh.unproject``` functions allows to switch between seeing a function in the LBO basis or on the complete shape.

The squared $L^2$ norm and $H^1_0$ norm can be computed via the ```mesh.l2_sqnorm``` and ```mesh.h1_sqnorm``` methods.

In [3]:
# By default does not use the intrinsic delaunay Laplacian
mesh1.process(k=3, intrinsic=False, verbose=True); #should replace by spectralnet

NameError: name 'mesh1' is not defined

In [11]:
# plot the third eigenfunction
plot_mesh(mesh1, mesh1.eigenvectors[:,2])

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-1.359730…

# 3 - Computing the functional map

**Loading data**

In [4]:
# mesh1 = TriMesh(np.load('data/lion-00_1e5.npy'))
# mesh2 = TriMesh(np.load('data/cat-00_1e5.npy'))
mesh1 = TriMesh('data/cat-00.off', area_normalize=True, center=False)
mesh2 = TriMesh('data/lion-00.off')

print(f'Mesh 1 : {mesh1.n_vertices:4d} vertices, {mesh1.n_faces:5d} faces\n'
      f'Mesh 2 : {mesh2.n_vertices:4d} vertices, {mesh2.n_faces:5d} faces')

double_plot(mesh1,mesh2)

Mesh 1 : 7207 vertices, 14410 faces
Mesh 2 : 5000 vertices,  9996 faces


**Computing descriptors**

In [5]:
from pyFM.functional import FunctionalMapping

process_params = {
    'n_ev': (35,35),  # Number of eigenvalues on source and Target
    'landmarks': np.loadtxt('data/landmarks.txt',dtype=int)[:5],  # loading 5 landmarks
    'subsample_step': 4,  # In order not to use too many descriptors
    'descr_type': 'WKS',  # WKS or HKS
}

model = FunctionalMapping(mesh1,mesh2)
model.preprocess(**process_params,verbose=True);


Computing Laplacian spectrum
Computing 200 eigenvectors
	Done in 4.03 s
Computing 200 eigenvectors
	Done in 2.23 s

Computing descriptors
	Normalizing descriptors

	150 out of 600 possible descriptors kept


In [14]:
# model.mesh1.load_eigen_vectors('eigen_vectors_lion_1e5.npy')
# model.mesh2.load_eigen_vectors('eigen_vectors_cat_1e5.npy')
# model.mesh1.load_eigen_vectors('data/emb_cat_off_35.npy')
# model.mesh2.load_eigen_vectors('data/emb_lion_off_35.npy')
# np.load('data/emb_cat_off_35.npy').shape

**Fitting the model**

$\newcommand{\RR}{\mathbb{R}}$
$\newcommand{\Ss}{\mathcal{S}}$
$\newcommand{\uargmin}[1]{\underset{#1}{\text{argmin}}\;}$
$\newcommand{\uargmax}[1]{\underset{#1}{\text{argmax}}\;}$
$\def\*#1{\mathbf{#1}}$

In pyFM, we always consider functional maps $\*C:\Ss_1\to\Ss_2$ and pointwise maps $T:\Ss_2\to\Ss_1$ going in opposite directions, with $\*C$ always going from shape 1 to shape 2 !

Optimization problem is
\begin{equation}
\uargmin{\*C\in\RR^{k_2\times k_1}} w_{descr}\|\*C\*A - \*B\|^2 + w_{lap}\|\*C\Delta_1 - \Delta_2\*C\|^2 + w_{\text{d- comm}}\sum_i \|\*C\Gamma_1^i - \Gamma_2^i\*C\|^2 + w_{\text{orient}}\sum_i \|\*C\Lambda_1^i - \Lambda_2^i\*C\|^2
\end{equation}

with $\Gamma_1^i$ and $\Gamma_2^i$ [multipliative operators](http://www.lix.polytechnique.fr/~maks/papers/fundescEG17.pdf) associated to the $i$-th descriptors, $\Lambda_1^i$ and $\Lambda_2^i$ [orientation preserving operators](https://arxiv.org/abs/1806.04455) associated to the $i$-th descriptors

In [15]:
fit_params = {
    'w_descr': 1e-1,
    'w_lap': 1e-2,
    'w_dcomm': 1e-1,
    'w_orient': 0
}

model.fit(**fit_params, verbose=True)

Computing commutativity operators
	Scaling LBO commutativity weight by 8.6e-09

Optimization :
	35 Ev on source - 35 Ev on Target
	Using 150 Descriptors
	Hyperparameters :
		Descriptors preservation :1.0e-01
		Descriptors commutativity :1.0e-01
		Laplacian commutativity :1.0e-02
		Orientation preservation :0.0e+00

	Task : CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH, funcall : 45, nit : 41, warnflag : 0
	Done in 0.49 seconds


**Visualizing the associated point to point map**

In [16]:
p2p_21 = model.get_p2p(n_jobs=1)
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21]
double_plot(mesh1,mesh2,cmap1,cmap2)

# 4 - Refining the Functional Map
```model.FM``` returns the current state of functional map. One can change which one is returned by using ```model.change_FM_type(FM_type)```, as one can see below. 

**ICP**

In [4]:
model.icp_refine(verbose=True)
p2p_21_icp = model.get_p2p()
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21_icp]
double_plot(mesh1,mesh2,cmap1,cmap2)

  0%|          | 0/10 [00:00<?, ?it/s]

**Zoomout**

In [5]:
model.change_FM_type('classic') # We refine the first computed map, not the icp-refined one
model.zoomout_refine(nit=15, step = 1, verbose=True)
print(model.FM.shape)
p2p_21_zo = model.get_p2p()
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21_zo]
double_plot(mesh1,mesh2,cmap1,cmap2)

AssertionError: Not enough eigenvectors on source :         25 are needed when 10 are provided

# Evaluating Results

In [6]:
import pyFM.eval

In [7]:
# Compute geodesic distance matrix on the cat mesh
A_geod = mesh1.get_geodesic(verbose=True)

  0%|          | 0/7207 [00:00<?, ?it/s]

In [ ]:
# Load an approximate ground truth map
gt_p2p = np.loadtxt('data/lion2cat',dtype=int)

acc_base = pyFM.eval.accuracy(p2p_21, gt_p2p, A_geod, sqrt_area=mesh1.sqrtarea)

acc_icp = pyFM.eval.accuracy(p2p_21_icp, gt_p2p, A_geod, sqrt_area=np.sqrt(mesh1.area))

acc_zo = pyFM.eval.accuracy(p2p_21_zo, gt_p2p, A_geod, sqrt_area=np.sqrt(mesh1.area))

print(f'Accuracy results\n'
      f'\tBasic FM : {1e3*acc_base:.2f}\n'
      f'\tICP refined : {1e3*acc_icp:.2f}\n'
      f'\tZoomOut refined : {1e3*acc_zo:.2f}\n')

: 

In [1]:
from pyFM.functional import FunctionalMapping
import meshplot as mp
import numpy as np

from pyFM.mesh import TriMesh


def plot_mesh(myMesh, cmap=None):
    mp.plot(myMesh.vertlist, myMesh.facelist, c=cmap)


def double_plot(myMesh1, myMesh2, cmap1=None, cmap2=None):
    d = mp.subplot(myMesh1.vertlist, myMesh1.facelist, c=cmap1, s=[2, 2, 0])
    mp.subplot(myMesh2.vertlist, myMesh2.facelist, c=cmap2, s=[2, 2, 1], data=d)


def visu(vertices):
    min_coord, max_coord = np.min(vertices, axis=0, keepdims=True), np.max(vertices, axis=0, keepdims=True)
    cmap = (vertices - min_coord) / (max_coord - min_coord)
    return cmap

mesh1 = TriMesh('data/cat-00.off', area_normalize=True, center=False)
mesh2 = TriMesh('data/lion-00.off')
# mesh1 = TriMesh(np.load('data/lion-00_1e5.npy'))
# mesh2 = TriMesh(np.load('data/cat-00_1e5.npy'))
process_params = {
    'n_ev': (10,10),  # Number of eigenvalues on source and Target
    'landmarks': np.loadtxt('data/landmarks.txt',dtype=int)[:5],  # loading 5 landmarks
    'subsample_step': 4,  # In order not to use too many descriptors
    'descr_type': 'WKS',  # WKS or HKS
    'k_process' : 10 # Number of eigenvalues to compute
}
model = FunctionalMapping(mesh1,mesh2)
model.preprocess(**process_params,verbose=True, is_point_cloud=False, intrinsic=False)
fit_params = {
    'w_descr': 1e-1,
    'w_lap': 1e-2,
    'w_dcomm': 1e-1,
    'w_orient': 0
}

model.fit(**fit_params, verbose=True)
p2p_21 = model.get_p2p(n_jobs=1)
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21]
double_plot(mesh1,mesh2,cmap1,cmap2)


Computing Laplacian spectrum
Computing 10 eigenvectors
eigenvalues sum : 315.40331138528313
Training SpectralNet:


  0%|          | 0/3 [00:00<?, ?it/s]

tensor([[ 7.2070e+03,  3.0518e-05,  9.1553e-04,  9.7656e-04,  2.3193e-03,
          6.0730e-03,  1.4343e-03,  1.1086e-02,  2.3499e-03,  1.3702e-02],
        [ 3.0518e-05,  7.2070e+03, -5.2490e-03, -1.9531e-03, -1.5488e-03,
         -1.0223e-02, -3.8147e-03, -2.9373e-04, -2.6119e-02, -1.3466e-02],
        [ 9.1553e-04, -5.2490e-03,  7.2070e+03,  1.0986e-03,  6.7139e-04,
         -5.9204e-03,  9.7656e-04, -2.1057e-03, -1.6479e-02,  5.0049e-03],
        [ 9.7656e-04, -1.9531e-03,  1.0986e-03,  7.2070e+03,  6.1035e-04,
         -3.6621e-04,  9.1553e-04,  1.4343e-03, -5.4932e-04,  4.2725e-04],
        [ 2.3193e-03, -1.5488e-03,  6.7139e-04,  6.1035e-04,  7.2070e+03,
         -2.5635e-03,  3.1738e-03,  2.9831e-03, -4.5471e-03,  4.1199e-03],
        [ 6.0730e-03, -1.0223e-02, -5.9204e-03, -3.6621e-04, -2.5635e-03,
          7.2070e+03, -3.0518e-05, -5.6648e-04, -3.4485e-03, -2.1362e-04],
        [ 1.4343e-03, -3.8147e-03,  9.7656e-04,  9.1553e-04,  3.1738e-03,
         -3.0518e-05,  7.2070e+0

Train Loss: 125.5873064, LR: 0.010000:  33%|███▎      | 1/3 [00:10<00:20, 10.07s/it]

tensor([[ 7.2070e+03,  1.5564e-03, -2.7847e-03, -3.1433e-03, -3.0823e-03,
         -3.2135e-02,  1.4069e-02,  6.5475e-02,  2.1362e-02,  2.3193e-03],
        [ 1.5564e-03,  7.2070e+03, -1.0452e-03, -9.1553e-04, -4.2725e-04,
          4.9133e-03,  8.7128e-03,  5.6381e-03,  2.1484e-02, -7.2021e-03],
        [-2.7847e-03, -1.0452e-03,  7.2070e+03,  7.6294e-05, -2.5177e-03,
          1.2527e-02, -1.5450e-03, -5.3749e-03, -4.9973e-03, -3.0457e-02],
        [-3.1433e-03, -9.1553e-04,  7.6294e-05,  7.2070e+03,  8.5449e-04,
          7.4158e-03, -1.5411e-03,  8.2397e-04, -6.3782e-03,  9.1553e-03],
        [-3.0823e-03, -4.2725e-04, -2.5177e-03,  8.5449e-04,  7.2070e+03,
          2.1362e-03,  2.2888e-03, -2.4948e-03, -1.2207e-03,  6.7139e-03],
        [-3.2135e-02,  4.9133e-03,  1.2527e-02,  7.4158e-03,  2.1362e-03,
          7.2070e+03,  8.5449e-04,  3.2043e-04,  5.1880e-04, -1.8311e-03],
        [ 1.4069e-02,  8.7128e-03, -1.5450e-03, -1.5411e-03,  2.2888e-03,
          8.5449e-04,  7.2070e+0

...

tensor([[ 5.0000e+03, -2.1052e-04,  1.0342e-03, -5.3366e-03, -3.0490e-03,
         -5.3032e-02,  3.5028e-02, -2.8799e-02, -4.4071e-02, -2.1813e-02],
        [-2.1052e-04,  5.0000e+03, -3.4237e-04, -4.3488e-04, -5.9128e-05,
         -1.5564e-03,  1.4687e-03,  2.6234e-02,  2.6909e-02, -1.0586e-02],
        [ 1.0342e-03, -3.4237e-04,  5.0000e+03,  1.9255e-03,  1.7805e-03,
          2.2516e-03,  4.0445e-03,  2.2733e-02,  9.7350e-03, -1.8954e-02],
        [-5.3366e-03, -4.3488e-04,  1.9255e-03,  5.0000e+03, -4.0340e-04,
          5.0564e-03, -7.5874e-03,  1.0522e-02,  1.7889e-02,  4.6539e-03],
        [-3.0490e-03, -5.9128e-05,  1.7805e-03, -4.0340e-04,  5.0000e+03,
          8.0681e-04, -1.3075e-03,  4.6587e-04, -4.8733e-04, -6.8779e-03],
        [-5.3032e-02, -1.5564e-03,  2.2516e-03,  5.0564e-03,  8.0681e-04,
          5.0000e+03, -3.3569e-04,  8.5449e-04,  1.4844e-03, -2.0065e-03],
        [ 3.5028e-02,  1.4687e-03,  4.0445e-03, -7.5874e-03, -1.3075e-03,
         -3.3569e-04,  5.0000e+0

Train Loss: 306.6562988, LR: 0.010000: 100%|██████████| 3/3 [00:10<00:00,  3.55s/it]


[[ 6.34175830e+03 -1.95872617e+04  3.07029883e+04 -1.15758604e+04
  -1.13996367e+04 -1.01210315e+03 -1.52318109e+05 -2.66094961e+04
  -3.28915312e+05 -2.57941250e+05]
 [-1.95872617e+04  7.48164531e+04 -1.41731406e+05  5.15566094e+04
   4.77978008e+04  2.08855098e+04  6.64446000e+05  7.94617109e+04
   1.23400475e+06  9.91544188e+05]
 [ 3.07029883e+04 -1.41731406e+05  3.07331969e+05 -1.05193508e+05
  -9.46207891e+04 -7.23044609e+04 -1.38969888e+06 -1.19190898e+05
  -2.44180450e+06 -1.97191900e+06]
 [-1.15758604e+04  5.15566094e+04 -1.05193508e+05  4.79307617e+04
   4.12803047e+04  1.38100273e+04  5.05444688e+05  5.00135508e+04
   7.73067625e+05  6.58875125e+05]
 [-1.13996367e+04  4.77978008e+04 -9.46207891e+04  4.12803047e+04
   4.33250586e+04  1.78753105e+04  4.40690094e+05  5.92526680e+04
   8.99528938e+05  7.10771438e+05]
 [-1.01210315e+03  2.08855098e+04 -7.23044609e+04  1.38100273e+04
   1.78753105e+04  8.10888047e+04  2.42810938e+05  2.62672637e+04
   7.62203000e+05  4.82699719e+05

c:\Users\Yuval\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(


	Done in 18.81 s

Computing descriptors
	Normalizing descriptors

	150 out of 600 possible descriptors kept
Computing commutativity operators
	Scaling LBO commutativity weight by 2.0e-12

Optimization :
	10 Ev on source - 10 Ev on Target
	Using 150 Descriptors
	Hyperparameters :
		Descriptors preservation :1.0e-01
		Descriptors commutativity :1.0e-01
		Laplacian commutativity :1.0e-02
		Orientation preservation :0.0e+00

	Task : CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL, funcall : 10, nit : 8, warnflag : 0
	Done in 0.07 seconds
